In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
#import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
red_wine = 'C://Users//vinic//Documents//Python//Machine Learning//Projetos UCI ML//Wine Quality//wine+quality//winequality-red.csv'
white_wine = 'C://Users//vinic//Documents//Python//Machine Learning//Projetos UCI ML//Wine Quality//wine+quality//winequality-white.csv'
data_red_wine = pd.read_csv(red_wine,header=0,sep=';')
data_white_wine = pd.read_csv(white_wine,header=0,sep=';')


In [1]:
data_white_wine.isna().sum()

NameError: name 'data_white_wine' is not defined

In [5]:
data_red_wine['quality'].unique()

array([5, 6, 7, 4, 8, 3], dtype=int64)

In [6]:
len(data_red_wine)

1599

In [7]:
data_white_wine['quality'].unique()

array([6, 5, 7, 8, 4, 3, 9], dtype=int64)

In [8]:
len(data_white_wine)

4898

## Train, Validation and test Data Sets

In [9]:
def scale_dataset(dataframe,oversample = False):
    X = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values
    
    scaler = StandardScaler()
    #Scale the number, so we dont have  huge discrepancy between columns, it affects the model
    X = scaler.fit_transform(X)
    
    #the difference between the len of the values must not be huge, so we have to scale it
    #oversample the one that has the least, taking more of the less class
    
    if oversample:
        ros = RandomOverSampler(random_state=42)
        X,y = ros.fit_resample(X,y)
        
    # concat 2 arrays, y is only one dimension so we have to make it 2 
    #in this function using -1 is the same as len(y)
    #same as concat in pandas put hstack in numpay
    df = np.hstack((X,np.reshape(y,(-1,1))))
    
    return df,X,y


In [10]:
## divide the data so we can do the one vs all model
### errado, tem que fazer isso depois de obter o test, train dataset
# red wine
#data_red_wine_one_vs_all = {}
#for i in np.sort(data_red_wine['quality'].unique()):
#    data_red_wine_one_vs_all[f'{i}'] = {}
#    temp = data_red_wine.copy()
#    temp['quality'] = (temp['quality'] == i).astype(int)
#    data_red_wine_one_vs_all[f'{i}']['data_frame'] = temp
#
## white wine
#data_white_wine_one_vs_all = {}
#for i in np.sort(data_white_wine['quality'].unique()):
#    data_white_wine_one_vs_all[f'{i}'] = {}
#    temp = data_white_wine.copy()
#    temp['quality'] = (temp['quality'] == i).astype(int)
#    data_white_wine_one_vs_all[f'{i}']['data_frame'] = temp


In [37]:
# red wine
data_red_wine_one_vs_all = {}
# 1. Split fixo único
# Split fixo único baseado no conjunto completo
red_wine_train, red_wine_valid, red_wine_test = np.split(
    data_red_wine.sample(frac=1, random_state=42), 
    [int(0.6 * len(data_red_wine)), int(0.8 * len(data_red_wine))]
)

# 2. Gerar datasets one-vs-all a partir do mesmo conjunto
for i in sorted(data_red_wine['quality'].unique()):
    # Transforma o label em binário
    train_bin = red_wine_train.copy()
    valid_bin = red_wine_valid.copy()
    test_bin = red_wine_test.copy()
    
    train_bin['quality'] = (train_bin['quality'] == i).astype(int)
    valid_bin['quality'] = (valid_bin['quality'] == i).astype(int)
    test_bin['quality'] = (test_bin['quality'] == i).astype(int)

    # Escala e armazena
    red_train, red_X_train, red_y_train = scale_dataset(train_bin, oversample=True)
    red_valid, red_X_valid, red_y_valid = scale_dataset(valid_bin, oversample=False)
    red_test, red_X_test, red_y_test = scale_dataset(test_bin, oversample=False)

    data_red_wine_one_vs_all[f'{i}'] = {
        'red_wine_train': red_train,
        'red_wine_X_train': red_X_train,
        'red_wine_y_train': red_y_train,
        'red_wine_valid':red_valid,
        'red_wine_X_valid':red_X_valid,
        'red_wine_y_valid':red_y_valid,
        'red_wine_test': red_test,
        'red_wine_X_test': red_X_test,
        'red_wine_y_test': red_y_test
    }

# white wine

data_white_wine_one_vs_all = {}

# 1. Split fixo único baseado no conjunto completo
white_wine_train, white_wine_test = train_test_split(
    data_white_wine, test_size=0.2, stratify=data_white_wine['quality'], random_state=42
)

# 2. Gerar datasets one-vs-all a partir do mesmo conjunto
for i in sorted(data_white_wine['quality'].unique()):
    # Transforma o label em binário
    train_bin = white_wine_train.copy()
    test_bin = white_wine_test.copy()
    
    train_bin['quality'] = (train_bin['quality'] == i).astype(int)
    test_bin['quality'] = (test_bin['quality'] == i).astype(int)

    # Escala e armazena
    white_train, white_X_train, white_y_train = scale_dataset(train_bin, oversample=True)
    white_test, white_X_test, white_y_test = scale_dataset(test_bin, oversample=False)

    data_white_wine_one_vs_all[f'{i}'] = {
        'white_wine_train': white_train,
        'white_wine_X_train': white_X_train,
        'white_wine_y_train': white_y_train,
        'white_wine_test': white_test,
        'white_wine_X_test': white_X_test,
        'white_wine_y_test': white_y_test
    }




c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


## Predict the Quality

# K-nearest neighbors - KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [13]:
data_red_wine['quality'].unique()

array([5, 6, 7, 4, 8, 3], dtype=int64)

In [14]:
data_red_wine['quality'].unique()+data_red_wine['quality'].unique()

array([10, 12, 14,  8, 16,  6], dtype=int64)

In [42]:
## red wine
class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_red_wine_one_vs_all['6']['red_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_red_wine['quality'].unique()):
    red_wine_knn_model = KNeighborsClassifier(n_neighbors=1)
    red_wine_knn_model.fit(data_red_wine_one_vs_all[f'{i}']['red_wine_X_train'],data_red_wine_one_vs_all[f'{i}']['red_wine_y_train'])
    #use the knn model to try to predict the test dataset
    red_wine_y_pred = red_wine_knn_model.predict(data_red_wine_one_vs_all[f'{i}']['red_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the Red Wine Knn para Qualidade->',i,'\n',classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred))
    class_rep[f'{i}'] = classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred,output_dict=True)
    
    number_samples_final += np.sum(red_wine_y_pred == 1)
    #print(red_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + red_wine_y_pred
    print("Number of FP",np.sum((~data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'] & red_wine_y_pred)==1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']+red_wine_y_pred)>1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']&red_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp



## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples = 0
calculo_numero_amostras  = 0

for i in np.sort(data_red_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]
#    ## pegar o numero de samples que deram true em algum momento
#    tp = class_rep[f'{i}']["1"]["support"]*class_rep[f'{i}']["1"]["recall"]
#    print("precision",class_rep[f'{i}']["1"]["precision"])
#    print("recall",class_rep[f'{i}']["1"]["recall"])
#    if class_rep[f'{i}']["1"]["precision"]> 0:
#        predicted_positive  = tp/(class_rep[f'{i}']["1"]["precision"])
#        print(predicted_positive)
#        calculo_numero_amostras += predicted_positive
#
for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_red_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_red_wine_one_vs_all['6']['red_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))

For the Red Wine Knn para Qualidade-> 3 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       317
           1       0.00      0.00      0.00         3

    accuracy                           0.99       320
   macro avg       0.50      0.50      0.50       320
weighted avg       0.98      0.99      0.98       320

Number of FP 1
Number of TP 0
Number of TP 0
1
Confusion Matrix para classe 3:
[[316   1]
 [  3   0]]
{'TN': 316, 'FP': 1, 'FN': 3, 'TP': 0}
For the Red Wine Knn para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       310
           1       0.25      0.20      0.22        10

    accuracy                           0.96       320
   macro avg       0.61      0.59      0.60       320
weighted avg       0.95      0.96      0.95       320

Number of FP 6
Number of TP 2
Number of TP 2
9
Confusion Matrix para classe 4:
[[304   6]
 [  8   2]]
{'TN': 304, 'FP': 6

In [43]:
## white wine

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_white_wine_one_vs_all['6']['white_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_white_wine['quality'].unique()):
    white_wine_knn_model = KNeighborsClassifier(n_neighbors=1)
    white_wine_knn_model.fit(data_white_wine_one_vs_all[f'{i}']['white_wine_X_train'],data_white_wine_one_vs_all[f'{i}']['white_wine_y_train'])
    #use the knn model to try to pwhiteict the test dataset
    white_wine_y_pred = white_wine_knn_model.predict(data_white_wine_one_vs_all[f'{i}']['white_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the White Wine Knn para Qualidade->',i,'\n',classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred))
    class_rep[f'{i}'] = classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,output_dict=True)
    number_samples_final += np.sum(white_wine_y_pred == 1)
    #print(white_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + white_wine_y_pred
    print("Number of FP",np.sum((~data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'] & white_wine_y_pred)==1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']+white_wine_y_pred)>1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']&white_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp    


## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0
for i in np.sort(data_white_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_white_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_white_wine_one_vs_all['6']['white_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))


For the White Wine Knn para Qualidade-> 3 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       976
           1       0.00      0.00      0.00         4

    accuracy                           0.99       980
   macro avg       0.50      0.50      0.50       980
weighted avg       0.99      0.99      0.99       980

Number of FP 1
Number of TP 0
Number of TP 0
1
Confusion Matrix para classe 3:
[[975   1]
 [  4   0]]
{'TN': 975, 'FP': 1, 'FN': 4, 'TP': 0}
For the White Wine Knn para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       947
           1       0.37      0.30      0.33        33

    accuracy                           0.96       980
   macro avg       0.67      0.64      0.66       980
weighted avg       0.96      0.96      0.96       980

Number of FP 17
Number of TP 10
Number of TP 10
28
Confusion Matrix para classe 4:
[[930  17]
 [ 23  10]]
{'TN': 930,

# Naive Bayes

In [18]:
from sklearn.naive_bayes import GaussianNB

In [44]:
## red wine

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_red_wine_one_vs_all['6']['red_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_red_wine['quality'].unique()):
    red_wine_nb_model = GaussianNB()
    red_wine_nb_model.fit(data_red_wine_one_vs_all[f'{i}']['red_wine_X_train'],data_red_wine_one_vs_all[f'{i}']['red_wine_y_train'])
    #use the nb model to try to predict the test dataset
    red_wine_y_pred = red_wine_nb_model.predict(data_red_wine_one_vs_all[f'{i}']['red_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the Red Wine NB para Qualidade->',i,'\n',classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred))
    class_rep[f'{i}'] = classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred,output_dict=True)
    number_samples_final += np.sum(red_wine_y_pred == 1)
    #print(red_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + red_wine_y_pred
    print("Number of FP",np.sum((~data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'] & red_wine_y_pred)==1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']+red_wine_y_pred)>1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']&red_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples)
    cm = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp    


## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0

for i in np.sort(data_red_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_red_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas: ",tpp+fpp)
valor = 0
for i in predicted_as_1_in_any_model:
    if i > 0:
        valor = valor + i
print(valor)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_red_wine_one_vs_all['6']['red_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))

For the Red Wine NB para Qualidade-> 3 
               precision    recall  f1-score   support

           0       0.99      0.86      0.92       317
           1       0.00      0.00      0.00         3

    accuracy                           0.85       320
   macro avg       0.49      0.43      0.46       320
weighted avg       0.98      0.85      0.91       320

Number of FP 45
Number of TP 0
Number of TP 0
980.0
Confusion Matrix para classe 3:
[[272  45]
 [  3   0]]
{'TN': 272, 'FP': 45, 'FN': 3, 'TP': 0}
For the Red Wine NB para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.99      0.65      0.78       310
           1       0.06      0.70      0.11        10

    accuracy                           0.65       320
   macro avg       0.52      0.68      0.45       320
weighted avg       0.96      0.65      0.76       320

Number of FP 108
Number of TP 7
Number of TP 7
980.0
Confusion Matrix para classe 4:
[[202 108]
 [  3   7]]
{'TN': 20

In [45]:
## white wine
class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_white_wine_one_vs_all['6']['white_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_white_wine['quality'].unique()):
    white_wine_nb_model = GaussianNB()
    white_wine_nb_model.fit(data_white_wine_one_vs_all[f'{i}']['white_wine_X_train'],data_white_wine_one_vs_all[f'{i}']['white_wine_y_train'])
    #use the nb model to try to pwhiteict the test dataset
    white_wine_y_pred = white_wine_nb_model.predict(data_white_wine_one_vs_all[f'{i}']['white_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the White Wine NB para Qualidade->',i,'\n',classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred))

    class_rep[f'{i}'] = classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,output_dict=True)
    number_samples_final += np.sum(white_wine_y_pred == 1)
    #print(white_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + white_wine_y_pred
    print("Number of FP",np.sum((~data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'] & white_wine_y_pred)==1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']+white_wine_y_pred)>1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']&white_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp


## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0
for i in np.sort(data_white_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_white_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_white_wine_one_vs_all['6']['white_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))

For the White Wine NB para Qualidade-> 3 
               precision    recall  f1-score   support

           0       1.00      0.91      0.95       976
           1       0.01      0.25      0.02         4

    accuracy                           0.91       980
   macro avg       0.50      0.58      0.49       980
weighted avg       0.99      0.91      0.95       980

Number of FP 83
Number of TP 1
Number of TP 1
84
Confusion Matrix para classe 3:
[[893  83]
 [  3   1]]
{'TN': 893, 'FP': 83, 'FN': 3, 'TP': 1}
For the White Wine NB para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.98      0.86      0.92       947
           1       0.13      0.61      0.21        33

    accuracy                           0.85       980
   macro avg       0.56      0.73      0.56       980
weighted avg       0.96      0.85      0.89       980

Number of FP 137
Number of TP 20
Number of TP 20
241
Confusion Matrix para classe 4:
[[810 137]
 [ 13  20]]
{'TN': 8

# SVM

In [47]:
from sklearn.svm import SVC

In [48]:
## red wine

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_red_wine_one_vs_all['6']['red_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0


for i in np.sort(data_red_wine['quality'].unique()):
    red_wine_svm_model = SVC()
    red_wine_svm_model.fit(data_red_wine_one_vs_all[f'{i}']['red_wine_X_train'],data_red_wine_one_vs_all[f'{i}']['red_wine_y_train'])
    #use the svm model to try to predict the test dataset
    red_wine_y_pred = red_wine_svm_model.predict(data_red_wine_one_vs_all[f'{i}']['red_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the Red Wine svm para Qualidade->',i,'\n',classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred,zero_division=True))
    class_rep[f'{i}'] = classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred,output_dict=True)
    
    number_samples_final += np.sum(red_wine_y_pred == 1)
    #print(red_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + red_wine_y_pred
    print("Number of FP",np.sum((~data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'] & red_wine_y_pred)==1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']+red_wine_y_pred)>1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']&red_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp

## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0

for i in np.sort(data_red_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_red_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_red_wine_one_vs_all['6']['red_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))



For the Red Wine svm para Qualidade-> 3 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       317
           1       0.00      0.00      0.00         3

    accuracy                           0.98       320
   macro avg       0.50      0.49      0.49       320
weighted avg       0.98      0.98      0.98       320

Number of FP 4
Number of TP 0
Number of TP 0
4
Confusion Matrix para classe 3:
[[313   4]
 [  3   0]]
{'TN': 313, 'FP': 4, 'FN': 3, 'TP': 0}
For the Red Wine svm para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.98      0.88      0.93       310
           1       0.12      0.50      0.20        10

    accuracy                           0.87       320
   macro avg       0.55      0.69      0.56       320
weighted avg       0.96      0.87      0.91       320

Number of FP 36
Number of TP 5
Number of TP 5
45
Confusion Matrix para classe 4:
[[274  36]
 [  5   5]]
{'TN': 274, 'FP':

In [49]:
## white wine

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_white_wine_one_vs_all['6']['white_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_white_wine['quality'].unique()):
    white_wine_svm_model = SVC(class_weight='balanced')
    white_wine_svm_model.fit(data_white_wine_one_vs_all[f'{i}']['white_wine_X_train'],data_white_wine_one_vs_all[f'{i}']['white_wine_y_train'])
    #use the svm model to try to pwhiteict the test dataset
    white_wine_y_pred = white_wine_svm_model.predict(data_white_wine_one_vs_all[f'{i}']['white_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the White Wine svm para Qualidade->',i,'\n',classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,zero_division=True))
    class_rep[f'{i}'] = classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,output_dict=True)

    number_samples_final += np.sum(white_wine_y_pred == 1)
    #print(white_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + white_wine_y_pred
    print("Number of FP",np.sum((~data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'] & white_wine_y_pred)==1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']+white_wine_y_pred)>1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']&white_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp
    
## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0
for i in np.sort(data_white_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_white_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Numero real de amostras no dataset teste: ",len(data_white_wine_one_vs_all['6']['white_wine_y_test']))
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))



For the White Wine svm para Qualidade-> 3 
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       976
           1       0.00      0.00      0.00         4

    accuracy                           0.98       980
   macro avg       0.50      0.49      0.49       980
weighted avg       0.99      0.98      0.99       980

Number of FP 17
Number of TP 0
Number of TP 0
17
Confusion Matrix para classe 3:
[[959  17]
 [  4   0]]
{'TN': 959, 'FP': 17, 'FN': 4, 'TP': 0}
For the White Wine svm para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.98      0.88      0.93       947
           1       0.13      0.52      0.21        33

    accuracy                           0.87       980
   macro avg       0.55      0.70      0.57       980
weighted avg       0.95      0.87      0.90       980

Number of FP 115
Number of TP 17
Number of TP 17
149
Confusion Matrix para classe 4:
[[832 115]
 [ 16  17]]
{'TN':

# Logistic Regression

In [50]:
from sklearn.linear_model import LogisticRegression

In [52]:
## red wine
class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_red_wine_one_vs_all['6']['red_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_red_wine['quality'].unique()):
    red_wine_lr_model = LogisticRegression()
    red_wine_lr_model.fit(data_red_wine_one_vs_all[f'{i}']['red_wine_X_train'],data_red_wine_one_vs_all[f'{i}']['red_wine_y_train'])
    #use the lr model to try to predict the test dataset
    red_wine_y_pred = red_wine_lr_model.predict(data_red_wine_one_vs_all[f'{i}']['red_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the Red Wine lr para Qualidade->',i,'\n',classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred))
    class_rep[f'{i}'] = classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred,output_dict=True)

    number_samples_final += np.sum(red_wine_y_pred == 1)
    #print(red_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + red_wine_y_pred
    print("Number of FP",np.sum((~data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'] & red_wine_y_pred)==1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']+red_wine_y_pred)>1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']&red_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp

## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0

for i in np.sort(data_red_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_red_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_red_wine_one_vs_all['6']['red_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))



For the Red Wine lr para Qualidade-> 3 
               precision    recall  f1-score   support

           0       0.99      0.96      0.97       317
           1       0.00      0.00      0.00         3

    accuracy                           0.95       320
   macro avg       0.50      0.48      0.49       320
weighted avg       0.98      0.95      0.97       320

Number of FP 13
Number of TP 0
Number of TP 0
13
Confusion Matrix para classe 3:
[[304  13]
 [  3   0]]
{'TN': 304, 'FP': 13, 'FN': 3, 'TP': 0}
For the Red Wine lr para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.99      0.82      0.90       310
           1       0.11      0.70      0.19        10

    accuracy                           0.82       320
   macro avg       0.55      0.76      0.54       320
weighted avg       0.96      0.82      0.87       320

Number of FP 56
Number of TP 7
Number of TP 7
76
Confusion Matrix para classe 4:
[[254  56]
 [  3   7]]
{'TN': 254, 'FP'

In [53]:
## white wine

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_white_wine_one_vs_all['6']['white_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_white_wine['quality'].unique()):
    white_wine_lr_model = LogisticRegression(max_iter=5000)
    white_wine_lr_model.fit(data_white_wine_one_vs_all[f'{i}']['white_wine_X_train'],data_white_wine_one_vs_all[f'{i}']['white_wine_y_train'])
    #use the lr model to try to pwhiteict the test dataset
    white_wine_y_pred = white_wine_lr_model.predict(data_white_wine_one_vs_all[f'{i}']['white_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the White Wine lr para Qualidade->',i,'\n',classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,zero_division=True))
    class_rep[f'{i}'] = classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,output_dict=True)

    number_samples_final += np.sum(white_wine_y_pred == 1)
    #print(white_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + white_wine_y_pred
    print("Number of FP",np.sum((~data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'] & white_wine_y_pred)==1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']+white_wine_y_pred)>1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']&white_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp

## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0
for i in np.sort(data_white_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_white_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_white_wine_one_vs_all['6']['white_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))


For the White Wine lr para Qualidade-> 3 
               precision    recall  f1-score   support

           0       1.00      0.73      0.84       976
           1       0.01      0.50      0.01         4

    accuracy                           0.73       980
   macro avg       0.50      0.61      0.43       980
weighted avg       0.99      0.73      0.84       980

Number of FP 266
Number of TP 2
Number of TP 2
268
Confusion Matrix para classe 3:
[[710 266]
 [  2   2]]
{'TN': 710, 'FP': 266, 'FN': 2, 'TP': 2}
For the White Wine lr para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.98      0.79      0.88       947
           1       0.09      0.61      0.16        33

    accuracy                           0.78       980
   macro avg       0.54      0.70      0.52       980
weighted avg       0.95      0.78      0.85       980

Number of FP 198
Number of TP 20
Number of TP 20
486
Confusion Matrix para classe 4:
[[749 198]
 [ 13  20]]
{'TN'

# Decision Tree Classification

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [55]:
## red wine

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_red_wine_one_vs_all['6']['red_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_red_wine['quality'].unique()):
    red_wine_dtc_model = DecisionTreeClassifier()
    red_wine_dtc_model.fit(data_red_wine_one_vs_all[f'{i}']['red_wine_X_train'],data_red_wine_one_vs_all[f'{i}']['red_wine_y_train'])
    #use the dtc model to try to predict the test dataset
    red_wine_y_pred = red_wine_dtc_model.predict(data_red_wine_one_vs_all[f'{i}']['red_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the Red Wine dtc para Qualidade->',i,'\n',classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred))
    class_rep[f'{i}'] = classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred,output_dict=True)

    number_samples_final += np.sum(red_wine_y_pred == 1)
    #print(red_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + red_wine_y_pred
    print("Number of FP",np.sum((~data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'] & red_wine_y_pred)==1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']+red_wine_y_pred)>1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']&red_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp

## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0

for i in np.sort(data_red_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_red_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_red_wine_one_vs_all['6']['red_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))


For the Red Wine dtc para Qualidade-> 3 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       317
           1       0.00      0.00      0.00         3

    accuracy                           0.98       320
   macro avg       0.50      0.50      0.50       320
weighted avg       0.98      0.98      0.98       320

Number of FP 2
Number of TP 0
Number of TP 0
2
Confusion Matrix para classe 3:
[[315   2]
 [  3   0]]
{'TN': 315, 'FP': 2, 'FN': 3, 'TP': 0}
For the Red Wine dtc para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       310
           1       0.09      0.10      0.10        10

    accuracy                           0.94       320
   macro avg       0.53      0.53      0.53       320
weighted avg       0.94      0.94      0.94       320

Number of FP 10
Number of TP 1
Number of TP 1
13
Confusion Matrix para classe 4:
[[300  10]
 [  9   1]]
{'TN': 300, 'FP':

In [58]:
## white wine

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_white_wine_one_vs_all['6']['white_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_white_wine['quality'].unique()):
    white_wine_dtc_model = DecisionTreeClassifier()
    white_wine_dtc_model.fit(data_white_wine_one_vs_all[f'{i}']['white_wine_X_train'],data_white_wine_one_vs_all[f'{i}']['white_wine_y_train'])
    #use the dtc model to try to pwhiteict the test dataset
    white_wine_y_pred = white_wine_dtc_model.predict(data_white_wine_one_vs_all[f'{i}']['white_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the White Wine dtc para Qualidade->',i,'\n',classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,zero_division=True))
    class_rep[f'{i}'] = classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,output_dict=True)
    
    number_samples_final += np.sum(white_wine_y_pred == 1)
    #print(white_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + white_wine_y_pred
    print("Number of FP",np.sum((~data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'] & white_wine_y_pred)==1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']+white_wine_y_pred)>1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']&white_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp

## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0
for i in np.sort(data_white_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_white_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Amostras NÃO CLassificadas",np.sum(predicted_as_1_in_any_model==0))

print("Numero real de amostras no dataset teste: ",len(data_white_wine_one_vs_all['6']['white_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))



For the White Wine dtc para Qualidade-> 3 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       976
           1       0.00      0.00      0.00         4

    accuracy                           0.99       980
   macro avg       0.50      0.50      0.50       980
weighted avg       0.99      0.99      0.99       980

Number of FP 2
Number of TP 0
Number of TP 0
2
Confusion Matrix para classe 3:
[[974   2]
 [  4   0]]
{'TN': 974, 'FP': 2, 'FN': 4, 'TP': 0}
For the White Wine dtc para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       947
           1       0.28      0.30      0.29        33

    accuracy                           0.95       980
   macro avg       0.63      0.64      0.63       980
weighted avg       0.95      0.95      0.95       980

Number of FP 26
Number of TP 10
Number of TP 10
38
Confusion Matrix para classe 4:
[[921  26]
 [ 23  10]]
{'TN': 921,

# Random Forest Classification

In [ ]:
## red wine

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_red_wine_one_vs_all['6']['red_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_red_wine['quality'].unique()):
    red_wine_rfc_model = RandomForestClassifier()
    red_wine_rfc_model.fit(data_red_wine_one_vs_all[f'{i}']['red_wine_X_train'],data_red_wine_one_vs_all[f'{i}']['red_wine_y_train'])
    #use the rfc model to try to predict the test dataset
    red_wine_y_pred = red_wine_rfc_model.predict(data_red_wine_one_vs_all[f'{i}']['red_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the Red Wine rfc para Qualidade->',i,'\n',classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred))

    class_rep[f'{i}'] = classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred,output_dict=True)
    
    number_samples_final += np.sum(red_wine_y_pred == 1)
    #print(red_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + red_wine_y_pred
    print("Number of FP",np.sum((~data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'] & red_wine_y_pred)==1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']+red_wine_y_pred)>1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']&red_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp

## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0

for i in np.sort(data_red_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_red_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Amostras NÃO CLassificadas",np.sum(predicted_as_1_in_any_model==0))
print("Numero real de amostras no dataset teste: ",len(data_red_wine_one_vs_all['6']['red_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))

c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\User

For the Red Wine rfc para Qualidade-> 3 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       317
           1       0.00      0.00      0.00         3

    accuracy                           0.99       320
   macro avg       0.50      0.50      0.50       320
weighted avg       0.98      0.99      0.99       320

Number of FP 0
Number of TP 0
Number of TP 0
0
Confusion Matrix para classe 3:
[[317   0]
 [  3   0]]
{'TN': 317, 'FP': 0, 'FN': 3, 'TP': 0}
For the Red Wine rfc para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       310
           1       0.50      0.10      0.17        10

    accuracy                           0.97       320
   macro avg       0.74      0.55      0.58       320
weighted avg       0.96      0.97      0.96       320

Number of FP 1
Number of TP 1
Number of TP 1
2
Confusion Matrix para classe 4:
[[309   1]
 [  9   1]]
{'TN': 309, 'FP': 1

c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\User

In [64]:
## white wine

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_white_wine_one_vs_all['6']['white_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_white_wine['quality'].unique()):
    white_wine_rfc_model = RandomForestClassifier()
    white_wine_rfc_model.fit(data_white_wine_one_vs_all[f'{i}']['white_wine_X_train'],data_white_wine_one_vs_all[f'{i}']['white_wine_y_train'])
    #use the rfc model to try to pwhiteict the test dataset
    white_wine_y_pred = white_wine_rfc_model.predict(data_white_wine_one_vs_all[f'{i}']['white_wine_X_test'])
    ## for better understanding of precition and recall
    ## https://en.wikipedia.org/wiki/Precision_and_recall

    print('For the White Wine rfc para Qualidade->',i,'\n',classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,zero_division=True))
    class_rep[f'{i}'] = classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,output_dict=True)
    
    number_samples_final += np.sum(white_wine_y_pred == 1)
    #print(white_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + white_wine_y_pred
    print("Number of FP",np.sum((~data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'] & white_wine_y_pred)==1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']+white_wine_y_pred)>1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']&white_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp

## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0
for i in np.sort(data_white_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_white_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Amostras NÃO CLassificadas",np.sum(predicted_as_1_in_any_model==0))

print("Numero real de amostras no dataset teste: ",len(data_white_wine_one_vs_all['6']['white_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)

print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))


For the White Wine rfc para Qualidade-> 3 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       976
           1       1.00      0.00      0.00         4

    accuracy                           1.00       980
   macro avg       1.00      0.50      0.50       980
weighted avg       1.00      1.00      0.99       980

Number of FP 0
Number of TP 0
Number of TP 0
0
Confusion Matrix para classe 3:
[[976   0]
 [  4   0]]
{'TN': 976, 'FP': 0, 'FN': 4, 'TP': 0}


c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


For the White Wine rfc para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       947
           1       0.75      0.18      0.29        33

    accuracy                           0.97       980
   macro avg       0.86      0.59      0.64       980
weighted avg       0.96      0.97      0.96       980

Number of FP 2
Number of TP 6
Number of TP 6
8
Confusion Matrix para classe 4:
[[945   2]
 [ 27   6]]
{'TN': 945, 'FP': 2, 'FN': 27, 'TP': 6}
For the White Wine rfc para Qualidade-> 5 
               precision    recall  f1-score   support

           0       0.85      0.88      0.86       689
           1       0.69      0.64      0.66       291

    accuracy                           0.81       980
   macro avg       0.77      0.76      0.76       980
weighted avg       0.80      0.81      0.81       980

Number of FP 85
Number of TP 187
Number of TP 187
280
Confusion Matrix para classe 5:
[[604  85]
 [104 187]]
{'TN': 

c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vinic\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Neural Network

In [65]:
import tensorflow as tf
from tensorflow.keras.layers import Input

# Red Whine NN

In [66]:
def train_model_red(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs,X_valid,y_valid):
    col = X_train.shape[1]
    num_classes = len(np.unique(y_train))
    nn_model = tf.keras.Sequential(
        [Input(shape=(col,)),
         tf.keras.layers.Dense(num_nodes,activation='relu'),
         tf.keras.layers.Dropout(dropout_prob),
         tf.keras.layers.Dense(num_nodes,activation='relu'),
         tf.keras.layers.Dropout(dropout_prob),
         tf.keras.layers.Dense(1,activation='sigmoid')
        ]
    )
    
    nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss='binary_crossentropy',metrics=['accuracy'])
    history = nn_model.fit(
        X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid,y_valid),verbose=0
        )
    return nn_model, history



In [67]:
#import numpy as np
#import tensorflow as tf
#from itertools import product
#from sklearn.metrics import classification_report, f1_score
#
#def find_best_param(X_train, y_train,X_valid,y_valid):
#    # Parâmetros para busca
#    num_nodes_list = [16, 32, 34]
#    dropout_probs = [0, 0.2]
#    learning_rates = [0.01, 0.005, 0.001]
#    batch_sizes = [32, 64, 128]
#    epochs = 100
#
#    # Melhor modelo baseado em validação
#    best_model = None
#    best_model_params = None
#    best_f1 = 0
#    best_accuracy = 0
#    best_val_loss = float('inf')
#
#    # Lista para armazenar resultados
#    results = []
#
#    # Loop pelos hiperparâmetros
#    for num_nodes, dropout_prob, lr, batch_size in product(num_nodes_list, dropout_probs, learning_rates, batch_sizes):
#        print(f"Testando: nodes={num_nodes}, dropout={dropout_prob}, lr={lr}, batch={batch_size}")
#
#        # Treina o modelo com os parâmetros atuais
#        model, history = train_model_red(
#            X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs,X_valid,y_valid
#        )
#
#        # Obtém a última perda, acurácia e F1-score no conjunto de validação
#        val_loss = history.history['val_loss'][-1]
#        val_accuracy = history.history['val_accuracy'][-1]
#
#
#        y_val_pred_probs = model.predict(X_valid)
#        y_val_pred = (y_val_pred_probs > 0.5).astype(int)  # Conversão para rótulo binário
#        val_f1 = f1_score(y_valid, y_val_pred, average='weighted')
#        
#        # Armazena os resultados
#        results.append({
#            "num_nodes": num_nodes, "dropout_prob": dropout_prob, "lr": lr, "batch_size": batch_size,
#            "val_loss": val_loss, "val_accuracy": val_accuracy, "val_f1": val_f1
#        })
#
#        # Atualiza o melhor modelo com base nas métricas
#        if (val_f1 > best_f1) or (val_f1 == best_f1 and val_accuracy > best_accuracy) or (val_f1 == best_f1 and val_loss < best_val_loss):
#            best_f1 = val_f1
#            best_accuracy = val_accuracy
#            best_val_loss = val_loss
#            best_model = model
#            best_model_params = {
#                "num_nodes": num_nodes, "dropout_prob": dropout_prob, "lr": lr, "batch_size": batch_size
#            }
#    return best_model_params

### AJUDA DO GPT





import numpy as np
import tensorflow as tf
from itertools import product
from sklearn.metrics import f1_score

def find_best_param(X_train, y_train, X_valid, y_valid):
    # Seed para reprodutibilidade
    tf.random.set_seed(42)
    np.random.seed(42)

    # Parâmetros para busca
    num_nodes_list = [16, 32, 34]
    dropout_probs = [0, 0.2]
    learning_rates = [0.01, 0.005, 0.001]
    batch_sizes = [32, 64, 128]
    epochs = 100

    # Inicialização do melhor modelo
    best_model_params = None
    best_f1 = 0
    best_accuracy = 0
    best_val_loss = float('inf')

    # Loop pelos hiperparâmetros
    for num_nodes, dropout_prob, lr, batch_size in product(num_nodes_list, dropout_probs, learning_rates, batch_sizes):
        print(f"\nTestando: nodes={num_nodes}, dropout={dropout_prob}, lr={lr}, batch={batch_size}")

        # Treina o modelo com os parâmetros atuais
        model, history = train_model_red(
            X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs, X_valid, y_valid
        )

        # Avaliação no conjunto de validação
        val_loss = history.history['val_loss'][-1]
        val_accuracy = history.history['val_accuracy'][-1]

        y_val_pred_probs = model.predict(X_valid)
        y_val_pred = (y_val_pred_probs > 0.5).astype(int)
        val_f1 = f1_score(y_valid, y_val_pred, average='weighted')

        # Mostra métricas da iteração
        print(f"F1: {val_f1:.4f}, Acc: {val_accuracy:.4f}, Loss: {val_loss:.4f}")

        # Atualiza o melhor conjunto de parâmetros
        if (val_f1 > best_f1) or (val_f1 == best_f1 and val_accuracy > best_accuracy) or (val_f1 == best_f1 and val_loss < best_val_loss):
            best_f1 = val_f1
            best_accuracy = val_accuracy
            best_val_loss = val_loss
            best_model_params = {
                "num_nodes": num_nodes,
                "dropout_prob": dropout_prob,
                "lr": lr,
                "batch_size": batch_size
            }

    return best_model_params




In [33]:
best_param_red_nn = {}
# Exibir os melhores parâmetros encontrados
for i in np.sort(data_red_wine['quality'].unique()):
    best_param_red_nn[f'{i}'] = find_best_param(data_red_wine_one_vs_all[f'{i}']['red_wine_X_train'], data_red_wine_one_vs_all[f'{i}']['red_wine_y_train'],data_red_wine_one_vs_all[f'{i}']['red_wine_X_valid'],data_red_wine_one_vs_all[f'{i}']['red_wine_y_valid'])
    



Testando: nodes=16, dropout=0, lr=0.01, batch=32
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.9897, Acc: 0.9906, Loss: 0.2508

Testando: nodes=16, dropout=0, lr=0.01, batch=64
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
F1: 0.9897, Acc: 0.9906, Loss: 0.3641

Testando: nodes=16, dropout=0, lr=0.01, batch=128
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.9897, Acc: 0.9906, Loss: 0.2114

Testando: nodes=16, dropout=0, lr=0.005, batch=32
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
F1: 0.9875, Acc: 0.9875, Loss: 0.1968

Testando: nodes=16, dropout=0, lr=0.005, batch=64
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.9897, Acc: 0.9906, Loss: 0.3412

Testando: nodes=16, dropout=0, lr=0.005, batch=128
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.9875, Acc: 0.9875, Loss: 0.2438

Testando: nodes=16, dropout=0, lr=0.001, batch=32
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.9897, Acc: 0.9906, Loss: 0.2105

Testando: nodes=16, dropout=0, lr=0.001, batch=64
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1: 0.9

In [34]:
print(best_param_red_nn)

{'3': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.001, 'batch_size': 64}, '4': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 128}, '5': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 128}, '6': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 64}, '7': {'num_nodes': 34, 'dropout_prob': 0.2, 'lr': 0.005, 'batch_size': 32}, '8': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.001, 'batch_size': 64}}


In [68]:
# como ja achei os melhores parametros, vou so jogar aq para agilizar

best_param_red_nn = {'3': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.001, 'batch_size': 64}, '4': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 128}, '5': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 128}, '6': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 64}, '7': {'num_nodes': 34, 'dropout_prob': 0.2, 'lr': 0.005, 'batch_size': 32}, '8': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.001, 'batch_size': 64}}


In [69]:
epochs = 100

In [70]:

class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_red_wine_one_vs_all['6']['red_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0

for i in np.sort(data_red_wine['quality'].unique()):
    model,history = train_model_red(data_red_wine_one_vs_all[f'{i}']['red_wine_X_train'], data_red_wine_one_vs_all[f'{i}']['red_wine_y_train'], best_param_red_nn[f'{i}']['num_nodes'], best_param_red_nn[f'{i}']['dropout_prob'], best_param_red_nn[f'{i}']['lr'], best_param_red_nn[f'{i}']['batch_size'],epochs,data_red_wine_one_vs_all[f'{i}']['red_wine_X_valid'],data_red_wine_one_vs_all[f'{i}']['red_wine_y_valid'])
    # Convert the TensorFlow tensor to a NumPy array
    print(f'Best Parameters for the {i} -> {best_param_red_nn[f"{i}"]}')
    red_wine_y_pred = model(data_red_wine_one_vs_all[f'{i}']['red_wine_X_test'])
    red_wine_y_pred = tf.convert_to_tensor(red_wine_y_pred).numpy()

    # Apply the thresholding and reshape
    red_wine_y_pred = (red_wine_y_pred > 0.5).astype(int).reshape(-1,)
    print('For the Red Wine NN para Qualidade->',i,'\n',classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred))
    class_rep[f'{i}'] = classification_report(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'],red_wine_y_pred,output_dict=True)
    
    number_samples_final += np.sum(red_wine_y_pred == 1)
    #print(red_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + red_wine_y_pred
    print("Number of FP",np.sum((~data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'] & red_wine_y_pred)==1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']+red_wine_y_pred)>1))
    print("Number of TP",np.sum((data_red_wine_one_vs_all[f'{i}']['red_wine_y_test']&red_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_red_wine_one_vs_all[f'{i}']['red_wine_y_test'], red_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp

## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0

for i in np.sort(data_red_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_red_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_red_wine_one_vs_all['6']['red_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))

Best Parameters for the 3 -> {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.001, 'batch_size': 64}
For the Red Wine NN para Qualidade-> 3 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       317
           1       0.00      0.00      0.00         3

    accuracy                           0.99       320
   macro avg       0.50      0.50      0.50       320
weighted avg       0.98      0.99      0.98       320

Number of FP 1
Number of TP 0
Number of TP 0
1
Confusion Matrix para classe 3:
[[316   1]
 [  3   0]]
{'TN': 316, 'FP': 1, 'FN': 3, 'TP': 0}
Best Parameters for the 4 -> {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 128}
For the Red Wine NN para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       310
           1       0.33      0.20      0.25        10

    accuracy                           0.96       320
   macro avg       0.65      0.59     

# White Wine NN

In [71]:
def train_model_white(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
    col = X_train.shape[1]
    #num_classes = len(np.unique(y_train))
    nn_model = tf.keras.Sequential(
        [Input(shape=(col,)),
         tf.keras.layers.Dense(num_nodes,activation='relu'),
         tf.keras.layers.Dropout(dropout_prob),
         tf.keras.layers.Dense(num_nodes,activation='relu'),
         tf.keras.layers.Dropout(dropout_prob),
         tf.keras.layers.Dense(1,activation='sigmoid')
        ]
    )
    
    nn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),loss='binary_crossentropy',metrics=['accuracy'])
    history = nn_model.fit(
        X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2,verbose=0
        )
    return nn_model, history



In [72]:
def find_best_param(X_train, y_train, X_valid, y_valid):
    # Seed para reprodutibilidade
    tf.random.set_seed(42)
    np.random.seed(42)

    # Parâmetros para busca
    num_nodes_list = [16, 32, 34]
    dropout_probs = [0, 0.2]
    learning_rates = [0.01, 0.005, 0.001]
    batch_sizes = [32, 64, 128]
    epochs = 100

    # Inicialização do melhor modelo
    best_model_params = None
    best_f1 = 0
    best_accuracy = 0
    best_val_loss = float('inf')

    # Loop pelos hiperparâmetros
    for num_nodes, dropout_prob, lr, batch_size in product(num_nodes_list, dropout_probs, learning_rates, batch_sizes):
        print(f"\nTestando: nodes={num_nodes}, dropout={dropout_prob}, lr={lr}, batch={batch_size}")

        # Treina o modelo com os parâmetros atuais
        model, history = train_model_white(
            X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs
        )

        # Avaliação no conjunto de validação
        val_loss = history.history['val_loss'][-1]
        val_accuracy = history.history['val_accuracy'][-1]

        y_val_pred_probs = model.predict(X_valid)
        y_val_pred = (y_val_pred_probs > 0.5).astype(int)
        val_f1 = f1_score(y_valid.astype(int), y_val_pred, average='weighted')

        # Mostra métricas da iteração
        print(f"F1: {val_f1:.4f}, Acc: {val_accuracy:.4f}, Loss: {val_loss:.4f}")

        # Atualiza o melhor conjunto de parâmetros
        if (val_f1 > best_f1) or (val_f1 == best_f1 and val_accuracy > best_accuracy) or (val_f1 == best_f1 and val_loss < best_val_loss):
            best_f1 = val_f1
            best_accuracy = val_accuracy
            best_val_loss = val_loss
            best_model_params = {
                "num_nodes": num_nodes,
                "dropout_prob": dropout_prob,
                "lr": lr,
                "batch_size": batch_size
            }

    return best_model_params

In [39]:
best_param_white_nn = {}
# Exibir os melhores parâmetros encontrados
for i in np.sort(data_white_wine['quality'].unique()):
    best_param_white_nn[f'{i}'] = find_best_param(data_white_wine_one_vs_all[f'{i}']['white_wine_X_train'], data_white_wine_one_vs_all[f'{i}']['white_wine_y_train'],data_white_wine_one_vs_all[f'{i}']['white_wine_X_test'],data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'])
    




Testando: nodes=16, dropout=0, lr=0.01, batch=32
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
F1: 0.9929, Acc: 1.0000, Loss: 0.0000

Testando: nodes=16, dropout=0, lr=0.01, batch=64
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
F1: 0.9918, Acc: 1.0000, Loss: 0.0000

Testando: nodes=16, dropout=0, lr=0.01, batch=128
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
F1: 0.9929, Acc: 1.0000, Loss: 0.0000

Testando: nodes=16, dropout=0, lr=0.005, batch=32
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
F1: 0.9929, Acc: 1.0000, Loss: 0.0000

Testando: nodes=16, dropout=0, lr=0.005, batch=64
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
F1: 0.9923, Acc: 1.0000, Loss: 0.0000

Testando: nodes=16, dropout=0, lr=0.005, batch=128
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
F1: 0.9913, Acc: 1.0000, Loss: 0.0000

Testando: nodes=16, dropout=0, lr=0.001, batch=32
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
F1: 0.9934, Acc: 1.0000, Loss: 0.0000

Testando: nodes=16, dropout=0, lr=0.001, batch=64
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
F1: 0.9918, Acc

In [40]:

print(best_param_white_nn)

{'3': {'num_nodes': 34, 'dropout_prob': 0.2, 'lr': 0.001, 'batch_size': 32}, '4': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.005, 'batch_size': 64}, '5': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 128}, '6': {'num_nodes': 32, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 128}, '7': {'num_nodes': 34, 'dropout_prob': 0.2, 'lr': 0.005, 'batch_size': 64}, '8': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 32}, '9': {'num_nodes': 32, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 32}}


In [73]:
## como ja achei os melhores parametros vou deixar aq pra agilizar

best_param_white_nn = {'3': {'num_nodes': 34, 'dropout_prob': 0.2, 'lr': 0.001, 'batch_size': 32}, '4': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.005, 'batch_size': 64}, '5': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 128}, '6': {'num_nodes': 32, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 128}, '7': {'num_nodes': 34, 'dropout_prob': 0.2, 'lr': 0.005, 'batch_size': 64}, '8': {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 32}, '9': {'num_nodes': 32, 'dropout_prob': 0, 'lr': 0.01, 'batch_size': 32}}

In [74]:
epochs = 100

In [75]:
class_rep = {}
number_samples_final = 0
predicted_as_1_in_any_model = np.zeros_like(data_white_wine_one_vs_all['6']['white_wine_y_test'], dtype=int)
fpp = 0
fnn = 0
tnn = 0
tpp = 0




for i in np.sort(data_white_wine['quality'].unique()):
    model,history = train_model_white(data_white_wine_one_vs_all[f'{i}']['white_wine_X_train'], data_white_wine_one_vs_all[f'{i}']['white_wine_y_train'], best_param_white_nn[f'{i}']['num_nodes'], best_param_white_nn[f'{i}']['dropout_prob'], best_param_white_nn[f'{i}']['lr'], best_param_white_nn[f'{i}']['batch_size'], epochs)
    # Convert the TensorFlow tensor to a NumPy array
    print(f'Best Parameters for the {i} -> {best_param_white_nn[f"{i}"]}')
    white_wine_y_pred = model.predict(data_white_wine_one_vs_all[f'{i}']['white_wine_X_test'])
    white_wine_y_pred = tf.convert_to_tensor(white_wine_y_pred).numpy()

    # Apply the thresholding and reshape
    white_wine_y_pred = (white_wine_y_pred > 0.5).astype(int).reshape(-1,)
    print('For the White Wine rfc para Qualidade->',i,'\n',classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,zero_division=True))

    class_rep[f'{i}'] = classification_report(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'],white_wine_y_pred,output_dict=True)
    
    number_samples_final += np.sum(white_wine_y_pred == 1)
    #print(white_wine_y_pred)
    predicted_as_1_in_any_model = predicted_as_1_in_any_model + white_wine_y_pred
    print("Number of FP",np.sum((~data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'] & white_wine_y_pred)==1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']+white_wine_y_pred)>1))
    print("Number of TP",np.sum((data_white_wine_one_vs_all[f'{i}']['white_wine_y_test']&white_wine_y_pred)==1))
    #print(predicted_as_1_in_any_model)
    print(number_samples_final)
    cm = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred)
    print(f"Confusion Matrix para classe {i}:\n{cm}")
    tn, fp, fn, tp = confusion_matrix(data_white_wine_one_vs_all[f'{i}']['white_wine_y_test'], white_wine_y_pred).ravel()
    print({'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp})
    fpp = fpp + fp
    fnn = fn + fnn
    tnn = tn + tnn
    tpp = tpp + tp


## tabela juntando os valores previstos como verdadeiros
macro_avgs = {"precision macro avg":0,"recall macro avg":0,"f1-score macro avg":0}
weighted_avg = {"precision weighted avg":0,"recall weighted avg":0,"f1-score weighted avg":0}
number_samples= 0
for i in np.sort(data_white_wine['quality'].unique()):
    macro_avgs["precision macro avg"] = macro_avgs["precision macro avg"] + class_rep[f'{i}']["1"]["precision"]
    weighted_avg["precision weighted avg"] = weighted_avg["precision weighted avg"] + class_rep[f'{i}']["1"]["precision"]*class_rep[f'{i}']["1"]["support"]
    macro_avgs["recall macro avg"] = macro_avgs["recall macro avg"] + class_rep[f'{i}']["1"]["recall"]
    weighted_avg["recall weighted avg"] = weighted_avg["recall weighted avg"] + class_rep[f'{i}']["1"]["recall"]*class_rep[f'{i}']["1"]["support"]    
    macro_avgs["f1-score macro avg"] = macro_avgs["f1-score macro avg"] + class_rep[f'{i}']["1"]["f1-score"]
    weighted_avg["f1-score weighted avg"] = weighted_avg["f1-score weighted avg"] + class_rep[f'{i}']["1"]["f1-score"]*class_rep[f'{i}']["1"]["support"]
    number_samples = number_samples + class_rep[f'{i}']["1"]["support"]

for i in macro_avgs:
    macro_avgs[i] = macro_avgs[i]/len(np.sort(data_white_wine['quality'].unique()))

for i in weighted_avg:
    weighted_avg[i] = weighted_avg[i]/number_samples

print("Para a analise total temos as medias:")
print("Para macro avg -> ",macro_avgs)
print("Para weighted avg -> ",weighted_avg)
print("Numero de amostrar que deram true em algum momento: ",number_samples_final)
print("Pela formula: ",calculo_numero_amostras)
print("Numero de Amostras que sao classificadas mais de uma vez: ",tpp+fpp)
print("Amostras repetidas pelo menos uma vez",np.sum(predicted_as_1_in_any_model>1))
print("Numero real de amostras no dataset teste: ",len(data_white_wine_one_vs_all['6']['white_wine_y_test']))
print('VALORES GERAIS -> FP: ',fpp,'FN: ',fnn,'TN ',tnn,"TP",tpp)
print(f"Overall-> Precision: {tpp/(tpp+fpp)} | Recall: {tpp/(tpp+fnn)}|F1-Score: {2/((1/(tpp/(tpp+fpp))) + (1/(tpp/(tpp+fnn))))}")
print("Overall Accuracy: ",(tpp+tnn)/(tpp+tnn+fpp+fnn))


Best Parameters for the 3 -> {'num_nodes': 34, 'dropout_prob': 0.2, 'lr': 0.001, 'batch_size': 32}
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For the White Wine rfc para Qualidade-> 3 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       976
           1       0.00      0.00      0.00         4

    accuracy                           0.99       980
   macro avg       0.50      0.50      0.50       980
weighted avg       0.99      0.99      0.99       980

Number of FP 2
Number of TP 0
Number of TP 0
2
Confusion Matrix para classe 3:
[[974   2]
 [  4   0]]
{'TN': 974, 'FP': 2, 'FN': 4, 'TP': 0}
Best Parameters for the 4 -> {'num_nodes': 34, 'dropout_prob': 0, 'lr': 0.005, 'batch_size': 64}
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
For the White Wine rfc para Qualidade-> 4 
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       947
           1       0.44      0.42      0.43        33

    ac

In [ ]:
### feito